# Julia: explore type hierarchy
With these two short recursive functions, one may look into Julia's type hierarchy.

In [1]:
function printsub(t::Type; first_iter = true, last=false, padding="")    
    if first_iter
        println(t)
        # don't print "| "
        last=true
        new_padding = ""
    else
        if last 
            println(padding * "┗ ",t)
            new_padding = padding * "  "
        else
            println(padding * "┣ ",t)
            new_padding = padding *  "┃ "
        end
    end
    
    st = subtypes(t)
        
    if ! isempty(st)
        for sub in st[1:lastindex(st)-1] 
            printsub(sub, first_iter=false, padding=new_padding)
        end
        printsub(st[end], first_iter=false, last=true, padding=new_padding)
    end
end

# accept abstract and concrete types that are subtypes of UnionAll or DataType,
# supertype(t) is defined for a argument of this type
function printsuper(t::Union{UnionAll,DataType}, res="$t")
    if supertype(t) != Any
        printsuper(supertype(t), "$(supertype(t)) -> $res")
    else
        println("Any -> $res")
    end
end;

To summarize some of the most important things about Julia's type system (for more refer to [Julia docs](https://docs.julialang.org/en/v1/manual/types/index.html)):
- it is dynamic, but you can indicate that certain values are of specific types
- hierarchical relationships are explicitly declared
    - all concrete types may only have abstract types as their supertypes
    - abstract and concrete types can be parameterized by other types
    - abstract types cannot be instantiated
- all values in Julia are true objects having a type
- the only type a value has is its actual type when the program is running
- only values, not variables, have types
- a primitive type is a concrete type whose data consists of plain old bits
- a composite type is a collection of named fields
- functions are not bundled with the objects they operate on
    - the types of all of a function's arguments are considered when selecting a method, rather than just the first one

In [2]:
printsub(Type)

Type
┣ Core.TypeofBottom
┣ DataType
┣ Union
┗ UnionAll


Parametric types like Array are a different kind of type called a UnionAll type. See the difference here:

In [3]:
typeof(Array)

UnionAll

In [4]:
typeof(Array)

UnionAll

In [5]:
typeof(Array{Int64})

UnionAll

In [6]:
typeof(Array{Int64,2})

DataType

In [7]:
printsuper(Array)

Any -> AbstractArray -> DenseArray -> Array


In [8]:
# Int is an alias for Int64
printsuper(Array{Int})

Any -> AbstractArray{Int64,N} where N -> DenseArray{Int64,N} where N -> Array{Int64,N} where N


In [9]:
# Only declared types (DataType) have unambiguous supertypes.
printsuper(Array{Int64,4})

Any -> AbstractArray{Int64,4} -> DenseArray{Int64,4} -> Array{Int64,4}


In [10]:
# DataType is its own type.
typeof(DataType)

DataType

In [11]:
printsub(AbstractString)

AbstractString
┣ String
┣ SubString
┣ SubstitutionString
┗ Test.GenericString


In [12]:
typeof(AbstractArray{String,1})

DataType

In [13]:
printsub(AbstractArray{String,1})

AbstractArray{String,1}
┣ AbstractRange{String}
┃ ┣ LinRange{String}
┃ ┣ OrdinalRange{String,S} where S
┃ ┃ ┣ AbstractUnitRange{String}
┃ ┃ ┗ StepRange{String,S} where S
┃ ┗ StepRangeLen{String,R,S} where S where R
┣ Base.LogicalIndex{String,A} where A<:(AbstractArray{Bool,N} where N)
┣ Base.ReinterpretArray{String,1,S,A} where A<:AbstractArray{S,1} where S
┣ Base.ReshapedArray{String,1,P,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where P<:AbstractArray
┣ Core.Compiler.AbstractRange{String}
┃ ┣ Core.Compiler.LinRange{String}
┃ ┣ Core.Compiler.OrdinalRange{String,S} where S
┃ ┃ ┣ Core.Compiler.AbstractUnitRange{String}
┃ ┃ ┗ Core.Compiler.StepRange{String,S} where S
┃ ┗ Core.Compiler.StepRangeLen{String,R,S} where S where R
┣ DenseArray{String,1}
┃ ┣ Array{String,1}
┃ ┣ Base.CodeUnits{String,S} where S<:AbstractString
┃ ┣ Random.UnsafeView{String}
┃ ┗ SharedArrays.SharedArray{String,1}
┣ JSON.Parser.PushVector{String,A} where A<

In [14]:
printsub(AbstractArray)

AbstractArray
┣ AbstractRange
┃ ┣ LinRange
┃ ┣ OrdinalRange
┃ ┃ ┣ AbstractUnitRange
┃ ┃ ┃ ┣ Base.OneTo
┃ ┃ ┃ ┣ Base.Slice
┃ ┃ ┃ ┗ UnitRange
┃ ┃ ┗ StepRange
┃ ┗ StepRangeLen
┣ Base.LogicalIndex
┣ Base.ReinterpretArray
┣ Base.ReshapedArray
┣ BitArray
┣ CartesianIndices
┣ Core.Compiler.AbstractRange
┃ ┣ Core.Compiler.LinRange
┃ ┣ Core.Compiler.OrdinalRange
┃ ┃ ┣ Core.Compiler.AbstractUnitRange
┃ ┃ ┃ ┣ Core.Compiler.OneTo
┃ ┃ ┃ ┣ Core.Compiler.Slice
┃ ┃ ┃ ┣ Core.Compiler.StmtRange
┃ ┃ ┃ ┗ Core.Compiler.UnitRange
┃ ┃ ┗ Core.Compiler.StepRange
┃ ┗ Core.Compiler.StepRangeLen
┣ Core.Compiler.BitArray
┣ Core.Compiler.LinearIndices
┣ DenseArray
┃ ┣ Array
┃ ┣ Base.CodeUnits
┃ ┣ Random.UnsafeView
┃ ┣ SharedArrays.SharedArray
┃ ┗ SuiteSparse.CHOLMOD.Dense
┣ JSON.Parser.PushVector
┣ LinearAlgebra.AbstractQ
┃ ┣ LinearAlgebra.QRCompactWYQ
┃ ┣ LinearAlgebra.QRPackedQ
┃ ┗ SuiteSparse.SPQR.QRSparseQ
┣ LinearAlgebra.AbstractTriangular
┃ ┣ LinearAlgebra.LowerTriangular
┃ ┣ LinearAlgebra.UnitLowerTriangular

In [15]:
printsub(Missing)

Missing


In [16]:
# Functions in Julia are first-class objects. 
# Abstract type of all functions.
printsuper(Function)

Any -> Function


In [17]:
isa(+, Function)

true

Thanks for reading.